In [191]:
from googletrans import Translator
import requests
import os
import json
import shutil

## Global variables

In [193]:
CAL_API_KEY='YOUR_API_KEY'
CAL_API_URL = 'https://api.calorieninjas.com/v1/nutrition'

## Get Chinese recipe 

In [194]:
recipe = '"五花肉": "600克","韓式泡菜": "200克","紅蘿蔔塊": "100克", "馬鈴薯塊": "100克","蔥白": "30克","蔥綠": "少許","蒜末": "50克","韓式辣醬": "1遲","米酒匙": "30毫升","雪白菇": "50克","鴻禧菇": "50克"'

### Translate recipe to English

In [195]:
translator = Translator()

translated_recipe = translator.translate(recipe, src="zh-TW", dest='en').text
print(translated_recipe)

"Pork Success": "600 grams", "Korean kimchi": "200 grams", "Carrot block": "100 grams", "potato pieces": "100 grams", "white white": "30 grams", "30 grams", "30 grams", "30 grams", "Onion green": "a little", "minced garlic": "50 grams", "Korean spicy sauce": "1 late", "rice wine": "30 ml", "Snow White mushroom": "50 grams", " Hongxi Mushroom ":" 50 grams "


### Text preprocessing

In [196]:
def format_ingredients(input_string):
        # print(input_string.split(', '))
        items = [item.split(':') for item in input_string.split(', ')]
        items = [item for item in items if len(item) == 2]

        ingredients_dict = dict(items)
        
        if "oil" in ingredients_dict:
            del ingredients_dict["oil"]
        output = ", ".join([f"{value.strip()} of {key.strip()}" for key, value in ingredients_dict.items()]).replace('"', "")
        return output
    

In [197]:
input_string = translated_recipe
formatted_translated_recipe = format_ingredients(input_string)
formatted_translated_recipe

'600 grams of Pork Success, 200 grams of Korean kimchi, 100 grams of Carrot block, 100 grams of potato pieces, 30 grams of white white, a little of Onion green, 50 grams of minced garlic, 1 late of Korean spicy sauce, 30 ml of rice wine, 50 grams of Snow White mushroom,  50 grams  of  Hongxi Mushroom '

## Call API to obtain total calories

In [161]:
headers = {'X-Api-Key': CAL_API_KEY}
payload = {'query': formatted_translated_recipe}

response = requests.get(CAL_API_URL, headers=headers, params=payload)

if response.status_code == requests.codes.ok:
    data = response.json()
    items = data['items']
    total_calories = 0
    print(items)

    for item in items:
        total_calories += item['calories']

    print("Total Calories:", total_calories)
else:
    print("Error:", response.status_code, response.text)

[{'name': 'flour', 'calories': 1087.8, 'serving_size_g': 300.0, 'fat_total_g': 3.0, 'fat_saturated_g': 0.5, 'protein_g': 30.7, 'sodium_mg': 6, 'potassium_mg': 319, 'cholesterol_mg': 0, 'carbohydrates_total_g': 230.9, 'fiber_g': 8.1, 'sugar_g': 0.8}, {'name': 'salt', 'calories': 0.0, 'serving_size_g': 2.0, 'fat_total_g': 0.0, 'fat_saturated_g': 0.0, 'protein_g': 0.0, 'sodium_mg': 767, 'potassium_mg': 0, 'cholesterol_mg': 0, 'carbohydrates_total_g': 0.0, 'fiber_g': 0.0, 'sugar_g': 0.0}, {'name': 'water', 'calories': 0.0, 'serving_size_g': 140.0, 'fat_total_g': 0.0, 'fat_saturated_g': 0.0, 'protein_g': 0.0, 'sodium_mg': 5, 'potassium_mg': 0, 'cholesterol_mg': 0, 'carbohydrates_total_g': 0.0, 'fiber_g': 0.0, 'sugar_g': 0.0}, {'name': 'water', 'calories': 0.0, 'serving_size_g': 50.0, 'fat_total_g': 0.0, 'fat_saturated_g': 0.0, 'protein_g': 0.0, 'sodium_mg': 2, 'potassium_mg': 0, 'cholesterol_mg': 0, 'carbohydrates_total_g': 0.0, 'fiber_g': 0.0, 'sugar_g': 0.0}, {'name': 'flour', 'calories':

In [162]:
class RecipeAnalyzer:
    def __init__(self, api_url, api_key):
        self.api_url = api_url
        self.api_key = api_key
        self.translator = Translator()

    def translate_recipe(self, recipe):
        recipe = recipe.replace(" ", "")
        # print('Original Recipe',recipe)
        translated_recipe = self.translator.translate(recipe, src="zh-TW", dest='en').text
        # print('Translated Recipe',translated_recipe)
        return translated_recipe
    
    def format_ingredients(self, input_string):
        # print(input_string.split(', '))
        items = [item.split(':') for item in input_string.split(', ')]
        items = [item for item in items if len(item) == 2]

        ingredients_dict = dict(items)
        
        if "oil" in ingredients_dict:
            del ingredients_dict["oil"]
        output = ", ".join([f"{value.strip()} of {key.strip()}" for key, value in ingredients_dict.items()]).replace('"', "")
        return output
    
    def get_calories(self, recipe):
        translated_recipe = self.translate_recipe(recipe)
        formatted_translated_recipe = self.format_ingredients(translated_recipe)

        headers = {'X-Api-Key': self.api_key}
        payload = {'query': formatted_translated_recipe}
        response = requests.get(self.api_url, headers=headers, params=payload)
        if response.status_code == requests.codes.ok:
            data = response.json()
            items = data['items']
            total_calories = 0
            for item in items:
                total_calories += item['calories']
            return total_calories
        else:
            print("Error:", response.status_code, response.text)
            return None

In [167]:
analyzer = RecipeAnalyzer(CAL_API_URL, CAL_API_KEY)
src_directory = 'icook_500_json'
dest_directory = 'icook_500_cal_json'

In [176]:
# 遍歷資料夾中的所有子資料夾
for subdir in [subd for subd in os.listdir(src_directory) if os.path.isdir(subdir_path) ]:
    subdir_path = os.path.join(src_directory, subdir)

    print(subdir)
    target_subdir_path = os.path.join(dest_directory, subdir)
    if not os.path.exists(target_subdir_path):
        os.makedirs(target_subdir_path)
    
    # 遍歷子資料夾中的所有 JSON 檔案
    for filename in os.listdir(subdir_path):
        if filename.endswith('.json'):
            file_path = os.path.join(subdir_path, filename)
            # 讀取 JSON 檔案中的資料
            with open(file_path, 'r', encoding="utf-8") as jsonfile:
                recipes_data = json.load(jsonfile)

            output_json_path = os.path.join(target_subdir_path, filename)
            with open(output_json_path, 'r', encoding="utf-8") as outfile:
                cal_data = json.load(outfile)

            # 創建一個空的字典，用於存儲 RecipeID 與卡路里的映射
            recipe_calories_mapping = {}

            # 遍歷每個食譜
            for recipe_data in recipes_data:
                recipe_id = recipe_data['RecipeID']
                chinese_recipe = ", ".join([f"{ingredient}: {quantity}" for ingredient, quantity in recipe_data['Ingredient'].items()])
                # 獲取卡路里
                total_calories = cal_data.get(recipe_id)
                servings = max(1, int(recipe_data["Servings"]))
                print(f'{total_calories=}')
                print(f'{servings=}')
                if total_calories is not None:
                    print(f'{(total_calories / servings)=}')
                    recipe_calories_mapping[recipe_id] = total_calories / servings
            
            output_json_path = os.path.join(target_subdir_path, filename)
            with open(output_json_path, 'w', encoding="utf-8") as outfile:
                json.dump(recipe_calories_mapping, outfile, indent=4)
            
            print(f"Saved matching recipes to {output_json_path}")

SubCategory_107_咖哩
total_calories=67.5
servings=2
(total_calories / servings)=33.75
total_calories=605.0
servings=1
(total_calories / servings)=605.0
total_calories=0.5
servings=4
(total_calories / servings)=0.125
total_calories=0.0
servings=4
(total_calories / servings)=0.0
total_calories=2.3333333333333335
servings=3
(total_calories / servings)=0.7777777777777778
total_calories=0.25
servings=4
(total_calories / servings)=0.0625
total_calories=0.25
servings=4
(total_calories / servings)=0.0625
total_calories=11.0
servings=2
(total_calories / servings)=5.5
total_calories=0.0
servings=4
(total_calories / servings)=0.0
total_calories=0.0
servings=5
(total_calories / servings)=0.0
total_calories=0.0
servings=4
(total_calories / servings)=0.0
total_calories=1.0
servings=3
(total_calories / servings)=0.3333333333333333
total_calories=35.5
servings=2
(total_calories / servings)=17.75
total_calories=3.0
servings=3
(total_calories / servings)=1.0
total_calories=0.0
servings=4
(total_calories /